In [1]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, VeraConfig, TaskType
import torch
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

checkpoint_name = "financial_sentiment_analysis_lora_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 5e-3 # 1e-2 is good
num_epochs = 5
batch_size = 8

/home/alex/work/peft/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# creating model
peft_config = VeraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=512, vera_dropout=0.1, projection_prng_key=0xABC, d_initial=0.1, save_projection=True, target_modules=["q", "k", "v"])

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

trainable params: 331,776 || all params: 1,229,913,088 || trainable%: 0.0269755646343687


PeftModelForSeq2SeqLM(
  (base_model): VeraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 1024)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 1024)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): Linear(
                    in_features=1024, out_features=1024, bias=False
                    (vera_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (vera_lambda_b): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 1024])
                    (vera_lambda_d): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 512])
                  )
                  (k): Linear(
                    in_features=1024, out_features=1024, bias=False
                    (vera_d

In [3]:
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Map: 100%|██████████| 227/227 [00:00<00:00, 57132.13 examples/s]


{'sentence': "Finnish plumbing and heating systems supplier Uponor 's net sales in continuing business operations decreased to EUR 249.1 mn in July-September 2008 , compared to EUR 262.1 mn in the third quarter of 2007 .",
 'label': 0,
 'text_label': 'negative'}

In [4]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset: 100%|██████████| 227/227 [00:00<00:00, 7340.39 examples/s]


In [5]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(
    [
        {"params": [p for n, p in model.named_parameters() if "vera_lambda_" in n], "lr": lr},
    ]
)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [6]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:03<00:00,  9.07it/s]


epoch=0: train_ppl=tensor(6.0480, device='cuda:0') train_epoch_loss=tensor(1.7997, device='cuda:0') eval_ppl=tensor(1.1687, device='cuda:0') eval_epoch_loss=tensor(0.1558, device='cuda:0')


100%|██████████| 29/29 [00:02<00:00,  9.69it/s]


epoch=1: train_ppl=tensor(1.1920, device='cuda:0') train_epoch_loss=tensor(0.1756, device='cuda:0') eval_ppl=tensor(1.1102, device='cuda:0') eval_epoch_loss=tensor(0.1045, device='cuda:0')


100%|██████████| 29/29 [00:03<00:00,  9.57it/s]


epoch=2: train_ppl=tensor(1.1507, device='cuda:0') train_epoch_loss=tensor(0.1404, device='cuda:0') eval_ppl=tensor(1.0972, device='cuda:0') eval_epoch_loss=tensor(0.0927, device='cuda:0')


100%|██████████| 29/29 [00:03<00:00,  9.34it/s]


epoch=3: train_ppl=tensor(1.1262, device='cuda:0') train_epoch_loss=tensor(0.1188, device='cuda:0') eval_ppl=tensor(1.0863, device='cuda:0') eval_epoch_loss=tensor(0.0827, device='cuda:0')


100%|██████████| 29/29 [00:02<00:00,  9.77it/s]

epoch=4: train_ppl=tensor(1.1281, device='cuda:0') train_epoch_loss=tensor(0.1205, device='cuda:0') eval_ppl=tensor(1.0824, device='cuda:0') eval_epoch_loss=tensor(0.0792, device='cuda:0')


In [7]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=94.7136563876652 % on the evaluation dataset
eval_preds[:10]=['neutral', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']
dataset['validation']['text_label'][:10]=['neutral', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']


In [8]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [9]:
ckpt = f"{peft_model_id}/adapter_model.bin"
!du -h $ckpt

5.5M	bigscience/mt0-large_PeftType.VERA_TaskType.SEQ_2_SEQ_LM/adapter_model.bin


In [10]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [11]:
model.eval()
model._add_forward_hooks()
i = 13
inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="pt")
print(dataset["validation"][text_column][i])
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

Finnish security and privacy software solutions developer Stonesoft Oyj said on January 7 , 2008 that the preliminary sales of its StoneGate products grew by 59 pct year-on-year to 3.6 mln euro ( $ 5.3 mln ) for the fourth quarter of 2007 .
{'input_ids': tensor([[   515, 143136,  20317,    305,  44357,   6296,  23760,  78709,  19401,
          20066,  20860,    385,   2426,    351,    259,   3829,    617,    259,
            261,   1508,    533,    287,    259,  92096,    276,  17689,    304,
           2476,  19401, 146643,  10371,   6998,    481,    455,   8266,  14442,
            270,   3721,    264,    444,    264,  12388,    288,  56492,    259,
          43022,   3789,    274,   1160,  72691,    259,  43022,    259,    271,
            332,    287,   9171,    807,  43095,    304,   1848,    259,    260,
              1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,